# Libraries

In [1]:
import scipy.io
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Load B0006 file

In [2]:
#File Path
data_file_path = 'C:/Users/ibrah/Desktop/Masters/Dissertation Progress/Datasets/Experimental Datasets/Battery Data Set/B0006.mat'

In [3]:
battery = 'B0006'
mat = scipy.io.loadmat(data_file_path, simplify_cells=True)

In [4]:
# create the dataframe with all battery's charge and discharge cycles

df = pd.DataFrame()

i = 0  # count all cycles
j = 0  # count only charge cycles
k = 0  # count only discharge cycles

for idx in range(len(mat[battery]['cycle'])):

    if mat[battery]['cycle'][idx]['type'] in ['charge', 'discharge']:

        i += 1

        # creates a string with the cycle timestamp
        time_string = '{}-{}-{} {}:{}:{}'.format(
            str(int(mat[battery]['cycle'][idx]['time'][0])),
            str(int(mat[battery]['cycle'][idx]['time'][1])),
            str(int(mat[battery]['cycle'][idx]['time'][2])),
            str(int(mat[battery]['cycle'][idx]['time'][3])),
            str(int(mat[battery]['cycle'][idx]['time'][4])),
            str(mat[battery]['cycle'][idx]['time'][5])
        )

        # reads cycle data
        df_aux = pd.DataFrame(mat[battery]['cycle'][idx]['data'])
        
        # if it is a charging cycle, fill in the Capacity column with the value of the capacity of the next discharging cycle, if any
        if mat[battery]['cycle'][idx]['type']=='charge':
            j += 1
            cycle_type_value = j
            if idx+1<len(mat[battery]['cycle']) and mat[battery]['cycle'][idx+1]['type']=='discharge':
                capacity_value = pd.DataFrame(mat[battery]['cycle'][idx+1]['data']).Capacity.mean()
            elif idx+2<len(mat[battery]['cycle']) and mat[battery]['cycle'][idx+2]['type']=='discharge':
                capacity_value = pd.DataFrame(mat[battery]['cycle'][idx+2]['data']).Capacity.mean()
            else:
                capacity_value = np.nan
            df_aux = (df_aux
                        .assign(Capacity = capacity_value)
                        .rename(columns={'Current_charge': 'Current', 'Voltage_charge': 'Voltage'})
                    )
        elif mat[battery]['cycle'][idx]['type']=='discharge':
            k += 1
            cycle_type_value = k
            df_aux = df_aux.rename(columns={'Current_load': 'Current', 'Voltage_load': 'Voltage'})

        # create auxiliary columns with the remaining cycle information
        df_aux = df_aux.assign(
                cycle = i,
                cycle_idx = idx,
                cycle_type = cycle_type_value,
                type = mat[battery]['cycle'][idx]['type'],
                ambient_temperature = mat[battery]['cycle'][idx]['ambient_temperature'],
                timestamp = pd.to_datetime(time_string)
            )

        # combine the data from this cycle with the data from previous cycles
        df = pd.concat([df, df_aux], axis=0)

# rearange the dataframe
# Starting from the indexes
df = df.reset_index()
# Columns ordering
cols = ['cycle', 'cycle_type', 'cycle_idx', 'index', 'type', 'ambient_temperature',
        'timestamp', 'Voltage_measured', 'Current_measured','Temperature_measured',
        'Current', 'Voltage', 'Time', 'Capacity']
df = df[cols]

In [5]:
df.head()

,cycle,cycle_type,cycle_idx,index,type,ambient_temperature,timestamp,Voltage_measured,Current_measured,Temperature_measured,Current,Voltage,Time,Capacity
0,1,1,0,0,charge,24,2008-04-02 13:08:17.921,3.864624,0.000082,24.682214,-0.001,-0.007,0.000,2.035338
1,1,1,0,1,charge,24,2008-04-02 13:08:17.921,3.469113,-4.059185,24.695407,-4.060,1.558,2.532,2.035338
2,1,1,0,2,charge,24,2008-04-02 13:08:17.921,3.994806,1.513750,24.711491,1.506,4.710,5.500,2.035338
3,1,1,0,3,charge,24,2008-04-02 13:08:17.921,4.005888,1.511389,24.739672,1.506,4.726,8.344,2.035338
4,1,1,0,4,charge,24,2008-04-02 13:08:17.921,4.012944,1.510817,24.753180,1.506,4.737,11.125,2.035338


In [6]:
#View only one registry per cycle (charge and discharge)
cycles = df.filter(['cycle','cycle_type','type','timestamp']).drop_duplicates()
cycles

,cycle,cycle_type,type,timestamp
0,1,1,charge,2008-04-02 13:08:17.921
789,2,1,discharge,2008-04-02 15:25:41.593
986,3,2,charge,2008-04-02 16:37:51.984
1926,4,2,discharge,2008-04-02 19:43:48.406
2122,5,3,charge,2008-04-02 20:55:40.812
...,...,...,...,...
583759,334,168,charge,2008-05-26 21:41:33.468
587251,335,167,discharge,2008-05-27 15:52:41.359
587549,336,169,charge,2008-05-27 17:53:59.765
591153,337,168,discharge,2008-05-27 20:45:42.125


# EDA

In [7]:
eda = df[['cycle','ambient_temperature',
        'timestamp', 'Voltage_measured', 'Current_measured','Temperature_measured',
        'Current', 'Voltage', 'Time', 'Capacity']].describe()

In [8]:
# Cycle Type (Charge and Discharge)
ct = df.groupby('type')['cycle_type'].describe().transpose()
ct

type,charge,discharge
count,541173.000000,50285.000000
mean,96.606471,88.125942
std,42.702483,45.699687
min,1.000000,1.000000
25%,62.000000,50.000000
50%,97.000000,88.000000
75%,133.000000,127.000000
max,170.000000,168.000000


In [9]:
# right join of eda and ct
eda = eda.join(ct)
eda


,cycle,ambient_temperature,timestamp,Voltage_measured,Current_measured,Temperature_measured,Current,Voltage,Time,Capacity,charge,discharge
count,591458.000000,591458.0,591458,591458.000000,591458.000000,591458.000000,591458.000000,591458.000000,591458.000000,591453.000000,541173.000000,50285.000000
mean,189.704094,24.0,2008-05-09 02:17:02.448848128,4.104234,0.359343,26.429154,0.619009,4.244536,4763.856934,1.488383,96.606471,88.125942
min,1.000000,24.0,2008-04-02 13:08:17.921000,-0.001422,-4.480834,22.984099,-4.480000,-0.007000,0.000000,1.153818,1.000000,1.000000
25%,119.000000,24.0,2008-05-01 14:41:41.577999872,4.154358,0.055642,24.501277,0.077000,4.235000,1928.175750,1.321579,62.000000,50.000000
50%,191.000000,24.0,2008-05-10 16:11:56.108999936,4.197002,0.186089,25.558404,0.266000,4.300000,4467.515500,1.447148,97.000000,88.000000
75%,264.000000,24.0,2008-05-18 23:53:33.780999936,4.199247,1.050413,27.611183,1.506000,4.635000,7503.824000,1.640135,133.000000,127.000000
max,338.000000,24.0,2008-05-28 11:09:42.046000,8.083344,1.530124,42.007540,1.999200,4.996000,10807.328000,2.035338,170.000000,168.000000
std,86.250140,0.0,NaN,0.224811,0.884314,2.794788,0.720501,0.733129,3147.859026,0.213705,42.702483,45.699687


In [10]:
# Number of registries per cycle type
df['type'].value_counts()

type
charge       541173
discharge     50285
Name: count, dtype: int64

In [11]:
eda.transpose()

,count,mean,min,25%,50%,75%,max,std
cycle,591458.0,189.704094,1.0,119.0,191.0,264.0,338.0,86.25014
ambient_temperature,591458.0,24.0,24.0,24.0,24.0,24.0,24.0,0.0
timestamp,591458,2008-05-09 02:17:02.448848128,2008-04-02 13:08:17.921000,2008-05-01 14:41:41.577999872,2008-05-10 16:11:56.108999936,2008-05-18 23:53:33.780999936,2008-05-28 11:09:42.046000,NaN
Voltage_measured,591458.0,4.104234,-0.001422,4.154358,4.197002,4.199247,8.083344,0.224811
Current_measured,591458.0,0.359343,-4.480834,0.055642,0.186089,1.050413,1.530124,0.884314
Temperature_measured,591458.0,26.429154,22.984099,24.501277,25.558404,27.611183,42.00754,2.794788
Current,591458.0,0.619009,-4.48,0.077,0.266,1.506,1.9992,0.720501
Voltage,591458.0,4.244536,-0.007,4.235,4.3,4.635,4.996,0.733129
Time,591458.0,4763.856934,0.0,1928.17575,4467.5155,7503.824,10807.328,3147.859026
Capacity,591453.0,1.488383,1.153818,1.321579,1.447148,1.640135,2.035338,0.213705


In [12]:
# Count how many rows (registries) exist for each cycle
df.cycle.value_counts(sort = False)

cycle
1       789
2       197
3       940
4       196
5       937
       ... 
334    3492
335     298
336    3604
337     300
338       5
Name: count, Length: 338, dtype: int64

# Save to csv

In [ ]:
df.to_csv("vars_b6.csv", index = False)